In [ ]:
#Installing required libraries
!pip install pandas geopandas matplotlib folium

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [ ]:
#Loading GeoJSON Files
wards = gpd.read_file("Wards_from_2022.geojson") #Loads file with D.C. Wards Boundaries
flood_risk = gpd.read_file("Floodplain_Base_Flood_Elevations.geojson") #Loads flood elevation file
storm_risk_areas = gpd.read_file("Storm_Surge_Risk_Areas.geojson") #Loads storm risk area file

#Checking data and outputting columns
print("Wards Columns:", wards.columns)
print("Flood Risk Columns:", flood_risk.columns)
print("Storm Risk Area Columns", storm_risk_areas.columns)

Wards Columns: Index(['WARD', 'NAME', 'REP_NAME', 'WEB_URL', 'REP_PHONE', 'REP_EMAIL',
       'REP_OFFICE', 'WARD_ID', 'LABEL', 'STUSAB', 'SUMLEV', 'GEOID',
       'GEOCODE', 'STATE', 'OBJECTID', 'GLOBALID', 'CREATED_DATE',
       'LAST_EDITED_DATE', 'SHAPEAREA', 'SHAPELEN', 'geometry'],
      dtype='object')
Flood Risk Columns: Index(['BFE_LN_ID', 'ELEV', 'LEN_UNIT', 'V_DATUM', 'SOURCE_CIT', 'DFIRM_ID',
       'VERSION_ID', 'GFID', 'GIS_ID', 'GLOBALID', 'CREATED', 'EDITED',
       'OBJECTID', 'SHAPELEN', 'geometry'],
      dtype='object')
Storm Risk Area Columns Index(['HES_ZONE', 'CONTOURLEN', 'GIS_ID', 'TARGET_FID', 'JOIN_COUNT',
       'GLOBALID', 'CREATED', 'EDITED', 'OBJECTID', 'SHAPEAREA', 'SHAPELEN',
       'geometry'],
      dtype='object')


In [ ]:
#Clipping flood zones to D.C. Wards
flood_in_wards = gpd.overlay(wards, flood_risk, how ='intersection')

#Calculating flood area per D.C. Ward
flood_in_wards['flood_area'] = flood_in_wards.geometry.area
flood_by_ward = flood_in_wards.groupby('WARD')['flood_area'].sum().reset_index()


/tmp/ipython-input-12-3018378247.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 527 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  flood_in_wards = gpd.overlay(wards, flood_risk, how ='intersection')
/tmp/ipython-input-12-3018378247.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  flood_in_wards['flood_area'] = flood_in_wards.geometry.area


In [ ]:
#Merging original wards to find the total area
wards['total_area'] = wards.geometry.area
result = wards.merge(flood_by_ward, on ='WARD', how ='left')
result['flood_pct'] = (result['flood_area'] / result['total_area']) * 100

#Sorting by most flood-prone/at-risk wards
result.sort_values('flood_pct', ascending = False, inplace = True)
print(result[['WARD', 'flood_pct']])

   WARD  flood_pct
0     8        NaN
1     6        NaN
2     7        NaN
3     2        NaN
4     1        NaN
5     5        NaN
6     3        NaN
7     4        NaN


/tmp/ipython-input-13-39371388.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wards['total_area'] = wards.geometry.area


In [ ]:
#Visualization
#fig, ax = plt.subplots(figsize = (10, 6))
#result.plot(column = 'flood_pct', legend = True, ax = ax, cmap = 'Reds', legend_kwds = {'label': '% Area at Flood Risk'})
#plt.title('Flood Risk by D.C. Ward')
#plt.show()

#Interactive Map
m = folium.Map(location = [38.895, -77.0369], zoom_start = 12)
folium.GeoJson(result[result['WARD'].isin(['7', '8'])], style_function = lambda x: {'fillColor': 'red' if x['properties']['flood_pct'] > 20 else 'orange'}).add_to(m)
m.save('flood_map.html')
m